<a href="https://colab.research.google.com/github/eylulpelinkilic/Clash-Royale-DSA210-Project/blob/main/Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test 1:

H0 (Null Hypothesis): Deck composition has no significant impact on match outcomes.

H1 (Alternative Hypothesis): Certain deck compositions significantly increase the probability of winning.

In [1]:
import pandas as pd
from scipy.stats import chi2_contingency

#load the match data
matches = pd.read_excel("matches.xlsx")

#count the most common 10 decks
top_decks = matches['playerDeck'].value_counts().head(10)

#label each deck as one of the top 10 or 'other'
def label_deck(deck_str):
    return deck_str if deck_str in top_decks.index else 'other'

matches['deckGroup'] = matches['playerDeck'].apply(label_deck)

#create a contingency table: deckGroup vs match outcome (win/loss)
contingency_table = pd.crosstab(matches['deckGroup'], matches['win'])

#chi-square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

#show results
print("Contingency Table (Deck vs Win):")
print(contingency_table)
print(f"\nChi-Square Statistic: {chi2_stat:.4f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-Value: {p_value:.4f}")

#hypothesis test result
alpha = 0.05
if p_value < alpha:
    print("\nConclusion: Deck composition has a statistically significant effect on match outcome. (Reject H0)")
else:
    print("\nConclusion: No significant effect of deck composition on match outcome was found. (Fail to reject H0)")


Contingency Table (Deck vs Win):
win                                                 False  True 
deckGroup                                                       
Bats,Mortar,Arrows,Boss Bandit,Cannon Cart,Ice ...      2      0
Goblin Barrel,Royal Recruits,Tesla,Dart Goblin,...      2      1
Goblin Barrel,Valkyrie,Dart Goblin,Goblin Gang,...      9     11
Hunter,Barbarians,Wizard,Bats,Witch,Miner,Skele...      1      1
Hunter,Barbarians,Wizard,Dart Goblin,Bats,Miner...     12     14
Hunter,Valkyrie,Skeleton King,Arrows,Firecracke...     10      9
Hunter,Valkyrie,Skeleton King,Arrows,Goblin Gan...      3      2
Hunter,Valkyrie,Skeleton King,Arrows,Ice Wizard...      1      2
Mega Knight,Witch,Electro Giant,P.E.K.K.A,Wizar...      5      5
Valkyrie,Hog Rider,Firecracker,Boss Bandit,Bats...     14     16
other                                                   8      4

Chi-Square Statistic: 4.7106
Degrees of Freedom: 10
P-Value: 0.9097

Conclusion: No significant effect of deck compositio

Test 2:

H0 (Null Hypothesis): Average elixir cost does not affect match performance.

H1 (Alternative Hypothesis): Extremely high or low elixir costs impact match performance significantly.

In [16]:
import pandas as pd
from scipy.stats import ttest_ind

#load the match data
df = pd.read_excel("matches.xlsx")

#create groups based on player average elixir
median_elixir = df["playerAvgElixir"].median()

#convert boolean to integers
low_elixir_group = low_elixir_group.astype(int)
high_elixir_group = high_elixir_group.astype(int)

#t-test
t_stat, p_value = ttest_ind(low_elixir_group, high_elixir_group)

print(f"\nT-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

#interpretation
if p_value < 0.05:
    print("Reject H0: Average elixir cost significantly affects match performance.")
else:
    print("Fail to reject H0: No significant effect of elixir cost on match performance.")


T-statistic: -0.1170
P-value: 0.9071
Fail to reject H0: No significant effect of elixir cost on match performance.


Test 3:

H0 (Null Hypothesis): The difference between the player's elixir cost and the opponent's elixir cost does not affect the match outcome.

H1 (Alternative Hypothesis): The elixir cost difference (higher/lower) significantly affects the probability of winning.

In [14]:
import pandas as pd
from scipy.stats import ttest_ind

#load the match data
df = pd.read_excel("matches.xlsx")

#compute the elixir difference between player and opponent
df["elixir_diff"] = df["playerAvgElixir"] - df["opponentAvgElixir"]

#split the dataset into wins and losses
won = df[df["win"] == True]["elixir_diff"]
lost = df[df["win"] == False]["elixir_diff"]

#perform an independent two-sample t-test
t_stat, p_value = ttest_ind(won, lost, equal_var=False)

#print the results
print("Elixir Cost Difference Hypothesis Test")
print("---------------------------------------")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Mean Elixir Diff (Win): {won.mean():.3f}")
print(f"Mean Elixir Diff (Loss): {lost.mean():.3f}")

#interpretation
if p_value < 0.05:
    print("\nResult: Reject H0 — elixir difference significantly affects match outcomes.")
else:
    print("\nResult: Fail to reject H0 — no significant effect of elixir difference on outcome.")


Elixir Cost Difference Hypothesis Test
---------------------------------------
t-statistic: 1.0347
p-value: 0.3028
Mean Elixir Diff (Win): 0.434
Mean Elixir Diff (Loss): 0.320

Result: Fail to reject H0 — no significant effect of elixir difference on outcome.


Test 4:

H0 (Null Hypothesis): The number of shared cards between the player and opponent has no effect on the match outcome.

H1 (Alternative Hypothesis): Sharing more cards with the opponent significantly affects the probability of winning.

In [11]:
import pandas as pd
from scipy.stats import ttest_ind

#load the match data
df = pd.read_excel("matches.xlsx")

#split decks into sets of card names
df["playerCards"] = df["playerDeck"].str.split(",")
df["opponentCards"] = df["opponentDeck"].str.split(",")

#count how many cards are shared between player and opponent
def count_common(row):
    return len(set(row["playerCards"]) & set(row["opponentCards"]))

df["common_cards"] = df.apply(count_common, axis=1)

#split by win/loss and run t-test
win_common = df[df["win"] == True]["common_cards"]
loss_common = df[df["win"] == False]["common_cards"]

t_stat, p_val = ttest_ind(win_common, loss_common, equal_var=False)

#print results
print("Hypothesis Test: Number of Shared Cards")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_val:.4f}")
print(f"Mean Shared Cards (Win): {win_common.mean():.3f}")
print(f"Mean Shared Cards (Loss): {loss_common.mean():.3f}")

if p_val < 0.05:
    print("Result: Reject H0 — Shared cards significantly affect outcome.")
else:
    print("Result: Fail to reject H0 — No significant effect of shared cards.")


Hypothesis Test: Number of Shared Cards
t-statistic: -2.5736
p-value: 0.0112
Mean Shared Cards (Win): 0.877
Mean Shared Cards (Loss): 1.433
Result: Reject H0 — Shared cards significantly affect outcome.


Test 5:

H0 (Null Hypothesis): The opponent’s average elixir cost has no impact on the match result.

H1 (Alternative Hypothesis): If the opponent has a lower average elixir cost, it significantly increases the chance of winning.

In [10]:
import pandas as pd
from scipy.stats import ttest_ind

#load the match data
df = pd.read_excel("matches.xlsx")

#split into two groups: win vs. loss
win_elixir = df[df["win"] == True]["opponentAvgElixir"]
loss_elixir = df[df["win"] == False]["opponentAvgElixir"]

#run independent two-sample t-test
t_stat, p_val = ttest_ind(win_elixir, loss_elixir, equal_var=False)

#print results
print("Hypothesis Test: Opponent's Average Elixir Cost")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_val:.4f}")
print(f"Mean Opponent Elixir (Win): {win_elixir.mean():.3f}")
print(f"Mean Opponent Elixir (Loss): {loss_elixir.mean():.3f}")

if p_val < 0.05:
    print("Result: Reject H0 — Opponent elixir cost significantly affects win rate.")
else:
    print("Result: Fail to reject H0 — No significant effect detected.")


Hypothesis Test: Opponent's Average Elixir Cost
t-statistic: -1.4647
p-value: 0.1455
Mean Opponent Elixir (Win): 3.517
Mean Opponent Elixir (Loss): 3.647
Result: Fail to reject H0 — No significant effect detected.
